In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
import openai
from openai import OpenAI
import os
openai.api_key = ""

template = """Chef's Hat is a four player card game where your goal is to discard all cards before the other players do.
At the beginning of the game every player receives a set of cards at their hand representing pizza ingredients.
The players discard the cards at the game board which represents a pizza.
To discard a card, the player follows a set of game rules which dictate which cards they can discard depending on the cards on the board.
Sometimes the player cannot discard a set of cards and they have to pass.
As the goal is to discard all the cards at hand, passing does not contribute to this goal.
During the game, if three out of the four players pass, the player that did not pass makes one pizza, removing all the cards on the board.
After making the pizza, this players gets to choose which cards of their hand will be on the board.

For each pizza, we calculate a set of metrics for each player.
These metrics represent the behavior of each player based on the actions they chose before the pizza was made.
The metrics are:
Attack: represents how effective was the action the player made,
by counting the amount of players that could not discard any card and then chose the action pass after this action.

Attack can have these values:
0: No players passed after the action.
1: One player passed after the action.
2: Two players passed after the action.
3: Three players passed after the action.

Vitality represents the number of non-pass actions the player did during that pizza.
0: Player didn't discard.
1: Player discarded once.
2: Player discarded twice.

Defense measures the amount of players that did the pass action before the current player discard action.
This measures the impact of a discard action of the current player in disrupting another player's discard action.
0 : The action the current players disrupted did not made any other player pass.
1 : The action the current players disrupted made one other player pass.
2 : The action the current players disrupted made two other players pass.


You are my assistant and I need you to help me describing the players behavior in the chef's hat game.
I'll give you the information in this format:
[Player_name, Attack, Defense, Vitality]

With these information and using only the context I gave you,
create a short, direct, concise and clear description of this player behavior.
Don't mistake discarding once with discarding one card.
In the response, you must not express any amount of cards, for example, don't say "a card", "one card" or "a single card".
Remember you are not talking about the cards, you are talking about the act of discarding.

You will reply in this format:
[Player name, Your reply]
"""

openai_api_key = ""
os.environ["OPENAI_API_KEY"] = openai_api_key

client = OpenAI()
messages=[{"role": "system", "content": template}]


In [ ]:
import pandas as pd

def save_results_to_excel(file_path):
    global results_df
    results_df.to_excel(file_path, index=False)

def read_games(file_path):

    # Read the CSV file into a DataFrame
    df = pd.read_excel(file_path)
    print(df)

    # Group by Match and Round, then process each group
    grouped = df.groupby(['Match', 'Round'])

    for name, group in grouped:
        # Keep only the last occurrence of each 'Source' within a round
        last_moves = group.drop_duplicates(subset='Source', keep='last')

        # List of all possible sources
        sources = ['RANDOM_Random_1', 'RANDOM_Random_2', 'RANDOM_Random_3', 'RANDOM_Random_4']

        for source in sources:
            source_data = last_moves[last_moves['Source'] == source]
            if not source_data.empty:
                row = source_data.iloc[0]  # Get the row for this source
                # Pass individual parameters to new_move
                new_move(row['Source'], row['Attack'], row['Defense'], row['Vitality'])
                print(new_move)

    save_results_to_excel('/content/result_df_descriptor.xlsx')

In [ ]:
import pandas as pd

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['Player_name', 'Attack', 'Defense', 'Vitality', 'ChatGPT_Response'])

def new_move(player_name, attack, defense, vitality):
    global results_df  # Referencing the global DataFrame

    # Assuming messages is a list that is initialized earlier
    messages = [
        {"role": "system", "content": template},
        {"role": "user", "content": f"{player_name}, {attack}, {defense}, {vitality}"}
    ]

    print("Messages:")
    print(messages)

    # Make sure you have initialized `client` properly
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
    )

    chatgpt_response = response.choices[0].message.content
    responses = chatgpt_response.split('\n')
    last_response = responses[-1]

    print("ChatGPT Last Response:")
    print(last_response)

    # Create a DataFrame for the new row
    new_row_df = pd.DataFrame([{
        'Player_name': player_name, 'Attack': attack, 'Defense': defense, 'Vitality': vitality,
        'ChatGPT_Response': last_response
    }], columns=results_df.columns)
    print("New Row DataFrame:")
    print(new_row_df)

    # Append the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
    print(results_df)

    # Optionally save results to CSV if needed
    # save_results_to_csv('/path/to/save.csv')

In [ ]:
read_games("/content/results_df.xlsx")

In [ ]:
# Now to process a response using the narrator template

commentator_template_prompt = """
You are Nova, a Chef's Hat card game narrator.
Chef's Hat is a four player card game where your goal is to discard all cards before the other players do.
At the beginning of the game every player receives a set of cards at their hand representing pizza ingredients.
The players discard the cards at the game board which represents a pizza.
To discard a card, the player follows a set of game rules which dictate which cards they can discard depending on the cards on the board.
Sometimes the player cannot discard a set of cards and they have to pass.
As the goal is to discard all the cards at hand, passing does not contribute to this goal.
During the game, if three out of the four players pass, the player that did not pass makes one pizza, removing all the cards on the board.
After making the pizza, this players gets to choose which cards of their hand will be on the board.

You will receive data in the following format: [Player name, move].
Narrate each move of each player.
Narrate how the other players reacted.
In this narration, pass and discard actions must be clearly described with the words "pass" or "passing" and "discard" or "discarding".
Your comment must be entertaining for spectators.
Don't mistake discarding once with discarding one card.
In the response, you must not express any amount of cards, for example, don't say "a card", "one card" or "a single card".
Remember you are not talking about the cards, you are talking about the act of discarding.



You will reply in this format:
[Player name, Your reply]
"""

def process_response_with_template(response_content, new_template_prompt):
    # Create messages list for the new prompt
    messages = [
        {"role": "system", "content": new_template_prompt},
        {"role": "user", "content": response_content}
    ]

    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the model
        messages=messages,
        temperature=0
    )

    # Extract and return the response content
    return response.choices[0].message.content

def process_all_responses(new_template_prompt):
    results_df  = pd.read_excel('/content/result_df_descriptor.xlsx')

    # Create a list to store processed responses
    processed_responses = []

    # Process each response in the DataFrame
    for index, row in results_df.iterrows():
        response_content = row['ChatGPT_Response']
        # Process the response with the new template
        processed_response = process_response_with_template(response_content, new_template_prompt)
        processed_responses.append(processed_response)

    # Add the processed responses as a new column in the DataFrame
    results_df['Processed_Response'] = processed_responses

    # Save the updated DataFrame to a CSV file
    results_df.to_excel('/content/result_df_commentator.xlsx', index=False)

In [ ]:
process_all_responses(commentator_template_prompt)

In [ ]:
!pip install openai
import pandas as pd
import openai
from openai import OpenAI
import os
openai.api_key = ""

In [ ]:
final_template_prompt = """
You are Nova, a Chef's Hat card game narrator.
You will receive four lines of data, all four describing the actions of four players and the consequences of their actions.
The information is mixed with some sentences generated for entertaining purposes.
Don't ignore the beginning of the sentences, it might include the name of the player you're referring to.

Get the information about the action of the player and the consequence for the other players in each line.
Pass and discard actions must be clearly described with the words "pass" or "passing" and "discard" or "discarding".
Use this information to create a single comment.
Identify in the comment which player you're talking about.
Your comment must be entertaining for spectators.

The rules for constructiong the comment are:
Don't repeat the previous comment.
Don't mistake discarding once with discarding one card.
In the response, you must not express any amount of cards, for example, don't say "a card", "one card" or "a single card".
Remember in this cases, you are not talking about the cards, you are talking about the act of discarding.
"""

In [ ]:
def process_pizzas(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path)  # Use pd.read_excel if working with an Excel file

    # Initialize list to store summaries
    pizza_summaries = []

    # Loop through the DataFrame in chunks of 4 (for each pizza)
    for i in range(0, len(df), 4):
        pizza_group = df.iloc[i:i+4]

        # Concatenate the 'Processed_Response' for all four players in the pizza group
        descriptions = "\n".join(pizza_group['Processed_Response'].tolist())

        # Ensure all four players' descriptions are collected before passing to OpenAI
        if len(pizza_group) == 4:
            processed_response = process_response_with_template(descriptions, final_template_prompt)

            # Make the API call
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # Specify the model
                messages=messages,
                temperature=0)

            # Extract the response text
            summary = processed_response

            # Add the summary to the list (for the whole pizza group)
            pizza_summaries.extend([summary] * 4)  # Repeat the summary for each player in the group
    # pizza_summaries.append("null")
    # pizza_summaries.append("null")
    # pizza_summaries.append("null")
    # Add the summaries back into the DataFrame as a new column
    df['Pizza_Summary'] = pizza_summaries

    # Save the updated DataFrame to an Excel file
    df.to_excel('processed_pizzas.xlsx', index=False)

In [ ]:
process_pizzas('/content/result_df_commentator.xlsx')

In [ ]:
!pip install openai
!pip install gtts
from gtts import gTTS
import pandas as pd
import openai
from openai import OpenAI
import os
openai.api_key = ""

In [ ]:
def process_xlsx_to_pizza(file_path):
    # Read the .xlsx file
    df = pd.read_excel(file_path)

    # Get the last column (assume it's the last one by index)
    last_column = df.iloc[:, -1]

    # Create a new DataFrame for pizzas
    pizza_data = {'Pizza': [], 'Narration': []}

    # Loop through the last column in steps of 4 (grouping by 4 lines)
    for i in range(0, len(last_column), 4):
        # Get the Pizza number
        pizza_number = f"Pizza {(i // 4) + 1}"

        # Get the fourth line of the current group (i.e., i+3)
        if i + 3 < len(last_column):
            fourth_line = last_column.iloc[i + 3]
        else:
            fourth_line = None  # Handle cases where lines are less than 4 at the end

        # Add the data to the pizza_data dictionary
        pizza_data['Pizza'].append(pizza_number)
        pizza_data['Narration'].append(fourth_line)

    # Create the final DataFrame
    pizza_df = pd.DataFrame(pizza_data)
    pizza_df.to_excel('pizzas_summary.xlsx', index=False)

    return pizza_df

In [ ]:
pizza_df = process_xlsx_to_pizza("/content/processed_pizzas.xlsx")

# Display the resulting DataFrame
print(pizza_df)

In [ ]:
def replace_player_names(text):
    player_mapping = {
        "RANDOM_Random_1": "Player number 1",
        "RANDOM_Random_2": "Player number 2",
        "RANDOM_Random_3": "Player number 3",
        "RANDOM_Random_4": "Player number 4"
    }

    # Replace player names using the mapping
    for old_name, new_name in player_mapping.items():
        text = text.replace(old_name, new_name)

    return text

def text_to_audio(df_pizzas, output_dir='audio_files'):
    # Read the .xlsx file
    df = pd.read_excel(df_pizzas)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through the DataFrame rows
    for index, row in df.iterrows():
        pizza_name = row['Pizza']
        text = row['Narration']

        if pd.isna(text) or not text.strip():
            # Skip if there's no text to convert
            print(f"Skipping {pizza_name} due to empty text.")
            continue

        # Replace player names in the text
        modified_text = replace_player_names(text)
        print("modified text")
        print(modified_text)

        # Convert the modified text to audio using gTTS
        tts = gTTS(text=modified_text, lang='en')

        # Save the audio file with a name like 'Pizza_1.mp3'
        audio_file = os.path.join(output_dir, f'{pizza_name}.mp3')
        tts.save(audio_file)

        print(f'Audio for {pizza_name} saved as {audio_file}')

In [ ]:
# Generate audio files
text_to_audio("/content/pizzas_summary.xlsx")